##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip
  
import os
import zipfile

local_zip = '/tmp/horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip = '/tmp/validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()
# 訓練用の馬の写真があるディレクトリ
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')

# 訓練用の人間の写真があるディレクトリ
train_human_dir = os.path.join('/tmp/horse-or-human/humans')

# 検証用の馬の写真があるディレクトリ　　　　
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')

# 検証用の人間の写真があるディレクトリ　　　
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

## 小さなモデルを初めから構築する

続ける前に、モデルの定義を開始しましょう。

まず初めに、tensorflowをインポートします。

In [ ]:
import tensorflow as tf

次に、前の例と同様に畳み込み層を追加し、最終結果をフラット化して、密結合層に与えます。

最後に、密結合層を追加します。

２クラスの分類問題、つまり、*２値分類問題* に直面しているため、ネットワークは [*sigmoid* 活性化関数](https://wikipedia.org/wiki/Sigmoid_function)で終了することに注意してください。これにより、ネットワークの出力は、０と１の間の単一のスカラになり、現在の画像がクラス１（クラス０ではなく）である確率をエンコードします。

In [ ]:
model = tf.keras.models.Sequential([
    # 入力形状が、300x300、3バイトの色という望ましい画像サイズであることに注目してください
    # これは最初の畳み込みです
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # ２番目の畳み込み
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 3番目の畳み込み
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 4番目の畳み込み
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 5番目の畳み込み
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # 結果をフラット化して、DNNに与えます
    tf.keras.layers.Flatten(),
    # 512ニューロンの隠れ層
    tf.keras.layers.Dense(512, activation='relu'),
    # 出力ニューロンは１つのみ。結果は０から1までの値を含み、０は一方のクラス（horses）を表し、１は他方のクラス（humans）を表します
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4),
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# すべての画像は1./255で再スケールされます
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

# train_datagenジェネレーターを使用した128のバッチで訓練用画像を流します
train_generator = train_datagen.flow_from_directory(
        '/tmp/horse-or-human/',  # これは訓練用画像のソースディレクトリです
        target_size=(300, 300),  # すべての画像は300x300にサイズ変更されます 
        batch_size=128,
        # binary_crossentropy損失関数を使用するため、２値ラベルが必要です
        class_mode='binary')

# validation_datagenジェネレーターを使用した32のバッチで検証用画像を流します
validation_generator = validation_datagen.flow_from_directory(
        '/tmp/validation-horse-or-human/',  # これは検証用画像のソースディレクトリです
        target_size=(300, 300),  # すべての画像は300x300にサイズ変更されます 
        batch_size=32,
        # binary_crossentropy損失関数を使用するため、２値ラベルが必要です
        class_mode='binary')

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=100,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()